Welcome to the ConTeXt-LangServer explorer.

This is a simple Jupyter-Lab notebook to allow us to explore the interaction between our evolving ConTeXt-LangServer and this simple [pygls](https://github.com/openlawlibrary/pygls) based LSP client.

To help us explore the LSP server we can use the:

- [LSP specification](https://microsoft.github.io/language-server-protocol/specifications/specification-current)
- [vscode-languageserver-node](https://github.com/microsoft/vscode-languageserver-node) ([Language Server Extension Guide](https://code.visualstudio.com/api/language-extensions/language-server-extension-guide))
  - [server](https://github.com/microsoft/vscode-languageserver-node/tree/main/server)
- [(python) lsprotocol](https://github.com/microsoft/lsprotocol/tree/main/packages/python) (The [types.py](https://github.com/microsoft/lsprotocol/blob/main/packages/python/lsprotocol/types.py) is used by pygls to provide the python class structures associated with the specification).
- [Pygls](https://github.com/openlawlibrary/pygls) [documentation](https://pygls.readthedocs.io/en/latest/) (provides the basic python LSP client)
- [Pygls Client class](https://github.com/openlawlibrary/pygls/blob/master/pygls/client.py) (provides a list of the methods which can be used to add LSP features as well as JSON-RPC commands to the client/server interface).
- [Pygls LangaugeClient(Client) class](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py) (provides a list of the LSP requests (both synchronous and asynchronous) which the LanguageClient implements).

It is very important to notice that we are currently using a pre-released version of pygls (since the *currently released* version, v1.0.2, does not *yet* implement the `LanguageClient` class).

We start by running the `setupClient.py` script to import pygls and create our LSP client

In [1]:
%run setupClient.py

Once we have a client we can use it to start the `context-langserver` (using JSON-RPC over `stdio`)

In [2]:
await client.start_io('context-langserver', '--stdio')

Before we do anything else we need to initialize the LSP client/server interface using the [InitializeParams](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#initializeParams) ([types](https://github.com/microsoft/lsprotocol/blob/main/packages/python/lsprotocol/types.py#L3026)) and [ClientCapabilities](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#clientCapabilities) ([types](https://github.com/microsoft/lsprotocol/blob/main/packages/python/lsprotocol/types.py#L6927)) classes.

In [3]:
response = await client.initialize_async(InitializeParams(
    capabilities=ClientCapabilities(
        workspace=WorkspaceClientCapabilities(
            configuration=True
        )
    )
))

The `ConTeXt-LangServer` responds with a (static) [InitializeResult](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#initializeResult) whose most important field is the [ServerCapabilities](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#serverCapabilities)

In [4]:
#print(yaml.dump(response))

Once we have the server's response, we need to send an [Initialized](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#initialized) notification using the [initialized](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py#L1838) method.

In [5]:
client.initialized(InitializedParams())

In [6]:
print(yaml.dump(registrations))

- !!python/object:lsprotocol.types.RegistrationParams
  registrations:
  - !!python/object:lsprotocol.types.Registration
    id: 7ae181ce-660b-4852-b6b1-3e633e342689
    method: workspace/didChangeConfiguration
    register_options: {}



In [7]:
response = await client.server_configuration_async(None)

In [8]:
print(yaml.dump(response))

hasConfigurationCapability: true
hasDiagnosticRelatedInformationCapability: false
hasWorkspaceFolderCapability: false



In response to the client's initialized notification, the server may send a number of dynamic [client/registerCapability](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#client_registerCapability) requests (with null repsonses).

Our language client captures these registration requests and places each one into the `registrations` array on a first come first appended basis.

In [8]:
#print(yaml.dump(registrations))

We play with the existing `onCompletion` request.

In [9]:
response = await client.text_document_completion_async(CompletionParams(
    text_document=TextDocumentIdentifier(
        uri="silly"
    ),
    position=Position(
        line=1,
        character=3
    )
))

In [10]:
print(yaml.dump(response))

- !!python/object:lsprotocol.types.CompletionItem
  additional_text_edits: null
  command: null
  commit_characters: null
  data: 1
  deprecated: null
  detail: null
  documentation: null
  filter_text: null
  insert_text: null
  insert_text_format: null
  insert_text_mode: null
  kind: !!python/object/apply:builtins.getattr
  - &id001 !!python/name:lsprotocol.types.CompletionItemKind ''
  - Text
  label: TypeScript
  label_details: null
  preselect: null
  sort_text: null
  tags: null
  text_edit: null
  text_edit_text: null
- !!python/object:lsprotocol.types.CompletionItem
  additional_text_edits: null
  command: null
  commit_characters: null
  data: 2
  deprecated: null
  detail: null
  documentation: null
  filter_text: null
  insert_text: null
  insert_text_format: null
  insert_text_mode: null
  kind: !!python/object/apply:builtins.getattr
  - *id001
  - Text
  label: JavaScript
  label_details: null
  preselect: null
  sort_text: null
  tags: null
  text_edit: null
  text_edit_

The *primary* objective of the `ConTeXt-LangServer` is to provide the editor with an overview of the *structure* of the ConTeXt document.

To do *this* we need to implement the [Document Symbols Request](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#textDocument_documentSymbol) in the `ConTeXt-LangServer`. (See the [LanguageClient::text_document_document_symbol](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py#L771) request)

However, before we can get the `DocumentSymbols` we first need to provide the `ConTeXt-LangServer` with the document.

To do *this* we need to inform the server of the document for which we want to get a structural overview. For an overview of this process, see the [TextDocument Synchronization](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#textDocument_synchronization) section of the LSP specification.

This means that the `ConTeXt-LangServer` needs to implement the [textDocument/didOpen](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#textDocument_didOpen),  [textDocument/didChange](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#textDocument_didChange) and [textDocument/didClose](https://microsoft.github.io/language-server-protocol/specifications/lsp/3.17/specification/#textDocument_didClose) requests (See the `LanguageClient` [text_document_did_open](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py#L1945), [text_document_did_change](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py#L1916) and [text_document_did_close](https://github.com/openlawlibrary/pygls/blob/master/pygls/lsp/client.py#L1927) respecitvely).

Finally we close the LSP client (and its associated server).

In [5]:
response = client.exit(None)